In [ ]:
from utils import *
import getpass

# from landsatxplore.earthexplorer import EarthExplorer
import tarfile
import boto3
from subprocess import run
import shlex
import time

keep_original_band_scenes = False
one_per_month = True
dir_suffix = ""
if (dir_suffix != "") and (not dir_suffix.endswith("/")):
    dir_suffix = dir_suffix + "/"

aws_session = rasterio.session.AWSSession(boto3.Session(), requester_pays=True)
# aws_session = rasterio.session.AWSSession(boto3.Session())

query_and_process = True
combined_path_rows = False

In [ ]:
wa_bbox = resize_bbox(
    read_kml_polygon("data/inputs_old/LANDSAT_8_127111/WA.kml")[1], 0.1
)
aoi_polys = kml_to_poly("data/inputs_old/aois.kml").geoms

bbox_list = [
    [67.45, -72.55, 67.55, -72.45],  # Amery bed rock
    [69.2, -68.1, 69.4, -67.9],  # Amery top
    [wa_bbox.left, wa_bbox.bottom, wa_bbox.right, wa_bbox.top],  # WA sand dunes
    *[list(p.bounds) for p in aoi_polys],  # AOI polygons
]

In [ ]:
platform = "LANDSAT_8"
r_channel = "red"
g_channel = "green"
b_channel = "blue"

aoi_index = 5

reference_month = "01"

print(bbox_list[aoi_index], "bbox")

if query_and_process:
    query = get_search_query(
        bbox_list[aoi_index],
        # start_date="",
        start_date="2013-01-01T00:00:00",
        end_date="2017-01-01T00:00:00",
        platform=platform,
        collection_category=None,
        collections=None,
    )
    # query = get_search_query(
    #     list(kml_to_poly("data/inputs_old/amery2.kml").bounds),
    #     # start_date="",
    #     start_date="1991-01-01T00:00:00",
    #     end_date="1993-12-01T00:00:00",
    #     platform=platform,
    #     collection_category=None,
    #     collections=None,
    #     cloud_cover=None,
    # )

    server_url = "https://landsatlook.usgs.gov/stac-server/search"
    features = query_stac_server(query, server_url)
    print(len(features), "features found")

In [ ]:
if query_and_process:
    scene_dict, scene_list = find_scenes_dict(
        features,
        one_per_month=False,
        # start_end_years=[2009, 2010],
        acceptance_list=["red", "green", "blue", "thumbnail"],
        remove_duplicate_times=True,
        duplicate_idx=1,
    )
    path_rows = list(scene_dict.keys())
    print(path_rows)
    dates = [list(scene_dict[pr].keys()) for pr in path_rows]
    print([len(d) for d in dates])

In [ ]:
path_row = "125109"
pr_list = ["125109", "128109"]

In [ ]:
if combined_path_rows:
    sn = combine_scene_dicts([scene_dict[pr] for pr in pr_list])
    data_dict = sn.copy()
    pr = "_".join(pr_list)
else:
    data_dict = scene_dict[path_row].copy()
    pr = path_row
print(pr)

In [ ]:
if query_and_process:
    closest_pair = get_pair_dict(data_dict, "closest", reference_month=reference_month)
    farthest_pair = get_pair_dict(
        data_dict, "farthest", reference_month=reference_month
    )
    print("Closest pair:")
    print(closest_pair[0])
    print(closest_pair[1])
    print("Farthest pair:")
    print(farthest_pair[0])
    print(farthest_pair[1])

    s3_list = [
        closest_pair[0]["thumbnail_alternate"],
        closest_pair[1]["thumbnail_alternate"],
        farthest_pair[1]["thumbnail_alternate"],
    ]
    outputs = []
    for url in s3_list:
        outputs.append(
            f"data/inputs/thumbnails/{dir_suffix}{platform}_{path_row}/{os.path.basename(url)}"
        )

    bucket = "usgs-landsat"
    download_files(bucket, s3_list, outputs, -1, is_async_download=False)

In [ ]:
if query_and_process:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    im0 = plt.imread(outputs[0])
    im1 = plt.imread(outputs[1])
    im2 = plt.imread(outputs[2])
    axes[0].imshow(im0)
    axes[1].imshow(im1)
    axes[2].imshow(im2)
    axes[0].set_title(os.path.basename(outputs[0]).replace("_thumb_small.jpeg", ""))
    axes[1].set_title(os.path.basename(outputs[1]).replace("_thumb_small.jpeg", ""))
    axes[2].set_title(os.path.basename(outputs[2]).replace("_thumb_small.jpeg", ""))
    plt.tight_layout()

In [ ]:
if query_and_process:
    date = list(data_dict.keys())[0]

    r_url = data_dict[date][0][r_channel]
    g_url = data_dict[date][0][g_channel]
    b_url = data_dict[date][0][b_channel]

    r_aws = data_dict[date][0][r_channel + "_alternate"]
    g_aws = data_dict[date][0][g_channel + "_alternate"]
    b_aws = data_dict[date][0][b_channel + "_alternate"]

    r_band_suffix = os.path.splitext(os.path.basename(r_url))[0].split("_")[-1]
    g_band_suffix = os.path.splitext(os.path.basename(g_url))[0].split("_")[-1]
    b_band_suffix = os.path.splitext(os.path.basename(b_url))[0].split("_")[-1]

    true_color_dir = f"data/inputs/{dir_suffix}{platform}_{pr}/true_color"
    os.makedirs(true_color_dir, exist_ok=True)

    true_color_ds_dir = f"data/inputs/{dir_suffix}{platform}_{pr}/true_color_ds"
    os.makedirs(true_color_ds_dir, exist_ok=True)

    closest_pair = get_pair_dict(data_dict, "closest", reference_month=reference_month)
    farthest_pair = get_pair_dict(
        data_dict, "farthest", reference_month=reference_month
    )

    pr_date_list = closest_pair + [farthest_pair[1]]
    for j, el in enumerate(pr_date_list):
        print(
            f"Now downloading and processing pairs for {el['scene_name']} and path_row: {pr}, scene {j + 1} of 3.",
            end="\r",
        )
        r_url = el[r_channel + "_alternate"]
        g_url = el[g_channel + "_alternate"]
        b_url = el[b_channel + "_alternate"]
        output_dir = (
            f"data/inputs/{dir_suffix}{platform}_{pr}/Originals/{el['scene_name']}"
        )
        os.makedirs(output_dir, exist_ok=True)
        r_output = os.path.join(output_dir, os.path.basename(r_url))
        g_output = os.path.join(output_dir, os.path.basename(g_url))
        b_output = os.path.join(output_dir, os.path.basename(b_url))
        r_img, r_meta = stream_scene_from_aws(r_url, aws_session)
        g_img, g_meta = stream_scene_from_aws(g_url, aws_session)
        b_img, b_meta = stream_scene_from_aws(b_url, aws_session)

        imgs = [r_img, g_img, b_img]
        outputs = [r_output, g_output, b_output]
        metas = [r_meta, g_meta, b_meta]
        for i, img in enumerate(imgs):
            with rasterio.open(outputs[i], "w", **metas[i]["profile"]) as ds:
                ds.write(img[0, :, :], 1)

        files = glob.glob(f"{output_dir}/**")
        r_band = list(filter(lambda f: f.endswith(f"_{r_band_suffix}.TIF"), files))[0]
        g_band = list(filter(lambda f: f.endswith(f"_{g_band_suffix}.TIF"), files))[0]
        b_band = list(filter(lambda f: f.endswith(f"_{b_band_suffix}.TIF"), files))[0]
        true_bands = [r_band, g_band, b_band]
        tc_file = f"{os.path.join(true_color_dir, os.path.basename(output_dir))}_TC.TIF"
        tc_file_ds = os.path.join(true_color_ds_dir, os.path.basename(tc_file))
        make_true_color_scene(true_bands, tc_file, gray_scale=True, averaging=True)
        downsample_dataset(tc_file, 0.2, tc_file_ds)

        el["local_path"] = tc_file
        el["local_path_ds"] = tc_file_ds

        if not keep_original_band_scenes:
            shutil.rmtree(
                f"data/inputs/{dir_suffix}{platform}_{pr}/Originals",
                ignore_errors=True,
            )

    cols = ["Reference", "Closest_target", "Farthest_target"]
    df = pd.DataFrame(
        {
            cols[i]: [
                el["local_path"],
                el["local_path_ds"],
            ]
            for i, el in enumerate(pr_date_list)
        },
        columns=cols,
    )
    df.to_csv(
        f"data/inputs/{dir_suffix}{platform}_{pr}/pairs.csv",
        index=False,
    )

In [ ]:
scene_df = pd.read_csv(f"data/inputs/{dir_suffix}{platform}_{pr}/pairs.csv")
ref_image = scene_df["Reference"][0]
tgt_images = [
    scene_df["Closest_target"][0],
    scene_df["Farthest_target"][0],
]
print("Reference image:", ref_image)
print("Closest target image:", tgt_images[0])
print("Farthest target image:", tgt_images[1])

#### Co_Register

In [ ]:
output_path = f"data/outputs/{dir_suffix}{platform}_{pr}/Co_Register"
_, shifts = co_register(
    ref_image,
    tgt_images,
    output_path=output_path,
    return_shifted_images=True,
    # use_overlap=False,
    # phase_corr_filter=False,
    # pahse_corr_valid_num_points=1,
    # of_dist_thresh=10,
    # band_number=2,
    # warping_filter=False,
    # laplacian_kernel_size=5,
)

#### AROSICS

In [ ]:
output_dir = f"data/outputs/{dir_suffix}{platform}_{pr}/AROSICS"
arosics(
    ref_image,
    tgt_images,
    output_dir,
)

#### Karios

In [ ]:
output_dir = f"data/outputs/{dir_suffix}{platform}_{pr}/Karios"
karios_executable = "/home/ubuntu/Coreg/karios/karios/karios.py"
karios(
    ref_image,
    tgt_images,
    output_dir,
    karios_executable,
)

In [ ]:
ref_time = datetime.strptime(os.path.basename(ref_image).split("_")[3], "%Y%m%d")
tgt_times = [
    datetime.strptime(os.path.basename(tgt).split("_")[3], "%Y%m%d")
    for tgt in tgt_images
]

In [ ]:
[(tgt_time - ref_time).days for tgt_time in tgt_times]